In [2]:
import pandas as pd
import numpy as np
import os

METHODS = ['knn', 'svm', 'naive_bayes', 'random_forest']
SETS = [
    'norm', 'norm_PCA80', 'norm_PCA95',
    'original', 'original_PCA80', 'original_PCA95',
    'stand', 'stand_PCA80', 'stand_PCA95'
]
METRICS_NAMES = ['Accuracy', 'Sensitivity', 'Specificity', 'Precision', 'F1_Score', 'FNR', 'FPR', 'AUC']
CLASSES = ['0', '1', '2']

## Función

In [ ]:
# ¿¿¿¿¿????????????????????????? TODO
# Para la métrica f1-score, una tabla resumen forma que en cada fila aparezca el
# rendimiento del método con cada conjunto de datos considerado y en cada columna
# aparezca cada conjunto de datos considerado. Cada celda tendrá el formato “media +-
# desviaciónTípica” (en Latex sería "media $\pm$ desviaciónTípica").

# TODO Genera las siguientes figuras:
# FN contra FP
# PR contra RC
# ACC contra Fm

# Así me gusta Carlos, métodos descriptivos
def haz_cosas(method, set):
    file_name = f"{method}_{set}_metrics.csv"
    full_path = os.path.join(os.getcwd(), "metrics", file_name)
    
    metrics = pd.read_csv(full_path, index_col="fold")
    
    # Calculamos la media de cada columna
    mean = metrics.mean().to_numpy()
    std = metrics.std().to_numpy()

    results = [f"{method}_{set}"]
    for i in range(0, mean.shape[0]):
        results.append(f"{mean[i]:.3f} +- {std[i]:.3f}")

    return results

## Generar csv

In [ ]:
try:
    os.mkdir(path="results")
except FileExistsError:
    pass

for method in METHODS:
    tabla_metodo = []
    
    for set in SETS:
        fila = haz_cosas(method, set)
        tabla_metodo.append(fila)

    full_path = os.path.join(os.getcwd(), "results", f"{method}.csv")

    df = pd.DataFrame(tabla_metodo, columns=["Set", *METRICS_NAMES])
    df.to_csv(full_path, index=False)
            

# Ensemble

In [4]:
def obtener_probabilidad(model_name, fold, set):
    """
    Obtienes el dataframe de probabilidades del modelo, fold y set especificado

    Args:
        model_name (str): Nombre del modelo (e.g., 'knn').
        fold (int): Número de la partición (e.g., 1).
        set_name (str): Versión del conjunto de datos (e.g., 'norm').

    Returns:
        pd.DataFrame: DataFrame con los resultados, o None si el archivo no existe.
    """
    # Obtenemos el path del csv y lo cargamos
    file_name = f"pred_{fold}_{set}_{model_name}.csv"
    full_path = os.path.join("predictions", file_name)

    predictions_df = pd.read_csv(full_path)

    return predictions_df

### Votación

In [ ]:
try:
    os.mkdir(path="ensemble_vote")
except FileExistsError:
    pass

for set in SETS:
    for fold in range(1,6):
        
        prediction_dfs = []

        for method in METHODS:
            # Metemos en una lista todos los dfs de probabilidades para ese fold y set
            probabilities = obtener_probabilidad(method, fold, set)
            if not probabilities.empty:
                prediction_dfs.append(probabilities)

        # Creamos el DataFrame base
        df_median = pd.DataFrame(0, index=prediction_dfs[0].index, columns=CLASSES, dtype=int)

        # Añadimos los votos
        for df in prediction_dfs:
            predicted_classes = df[CLASSES].idxmax(axis=1)

            for index, clase in predicted_classes.items():
                df_median.loc[index, clase] += 1

        # Añadimos la columna que recoja la decisión del Ensemble
        df_median['Ensemble_Decision'] = df_median[CLASSES].idxmax(axis=1)

        # Guardamos en csv el DataFrame de votos
        file_name = f"votes_fold{fold}_{set}.csv"
        full_path = os.path.join("ensemble_vote", file_name)
            
        df_median.to_csv(full_path, index=True, index_label='Index')
        print(f"Generado: {full_path} con {len(prediction_dfs)} modelos.")        

Generado: ensemble_vote\votes_fold1_norm.csv con 4 modelos.
Generado: ensemble_vote\votes_fold2_norm.csv con 4 modelos.
Generado: ensemble_vote\votes_fold3_norm.csv con 4 modelos.
Generado: ensemble_vote\votes_fold4_norm.csv con 4 modelos.
Generado: ensemble_vote\votes_fold5_norm.csv con 4 modelos.
Generado: ensemble_vote\votes_fold1_norm_PCA80.csv con 4 modelos.
Generado: ensemble_vote\votes_fold2_norm_PCA80.csv con 4 modelos.
Generado: ensemble_vote\votes_fold3_norm_PCA80.csv con 4 modelos.
Generado: ensemble_vote\votes_fold4_norm_PCA80.csv con 4 modelos.
Generado: ensemble_vote\votes_fold5_norm_PCA80.csv con 4 modelos.
Generado: ensemble_vote\votes_fold1_norm_PCA95.csv con 4 modelos.
Generado: ensemble_vote\votes_fold2_norm_PCA95.csv con 4 modelos.
Generado: ensemble_vote\votes_fold3_norm_PCA95.csv con 4 modelos.
Generado: ensemble_vote\votes_fold4_norm_PCA95.csv con 4 modelos.
Generado: ensemble_vote\votes_fold5_norm_PCA95.csv con 4 modelos.
Generado: ensemble_vote\votes_fold1_orig

### Media

In [ ]:
try:
    os.mkdir(path="ensemble_avg")
except FileExistsError:
    pass

for set in SETS:
    for fold in range(1,6):
        
        prediction_dfs = []

        for method in METHODS:
            # Metemos en una lista todos los dfs de probabilidades para ese fold y set
            probabilities = obtener_probabilidad(method, fold, set)
            if not probabilities.empty:
                prediction_dfs.append(probabilities)

        # Creamos el DataFrame base
        df_median = pd.DataFrame(0, index=prediction_dfs[0].index, columns=CLASSES, dtype=int)

        number_methods = len(METHODS)

        # Sumamos todos los DataFrames
        for df in prediction_dfs:
            df_median = df_median.add(df, fill_value=0)

        # Calculamos la media y redondeamos
        df_median = df_median / number_methods
        df_median[CLASSES] = df_median[CLASSES].round(3)   

        # Añadimos la columna que recoja la decisión del Ensemble
        df_median['Ensemble_Decision'] = df_median[CLASSES].idxmax(axis=1)

        # Guardamos en csv el DataFrame de votos
        file_name = f"avg_fold{fold}_{set}.csv"
        full_path = os.path.join("ensemble_avg", file_name)
            
        df_median.to_csv(full_path, index=True, index_label='Index')
        print(f"Generado: {full_path} con {len(prediction_dfs)} modelos.")        

Generado: ensemble_avg\avg_fold1_norm.csv con 4 modelos.
Generado: ensemble_avg\avg_fold2_norm.csv con 4 modelos.
Generado: ensemble_avg\avg_fold3_norm.csv con 4 modelos.
Generado: ensemble_avg\avg_fold4_norm.csv con 4 modelos.
Generado: ensemble_avg\avg_fold5_norm.csv con 4 modelos.
Generado: ensemble_avg\avg_fold1_norm_PCA80.csv con 4 modelos.
Generado: ensemble_avg\avg_fold2_norm_PCA80.csv con 4 modelos.
Generado: ensemble_avg\avg_fold3_norm_PCA80.csv con 4 modelos.
Generado: ensemble_avg\avg_fold4_norm_PCA80.csv con 4 modelos.
Generado: ensemble_avg\avg_fold5_norm_PCA80.csv con 4 modelos.
Generado: ensemble_avg\avg_fold1_norm_PCA95.csv con 4 modelos.
Generado: ensemble_avg\avg_fold2_norm_PCA95.csv con 4 modelos.
Generado: ensemble_avg\avg_fold3_norm_PCA95.csv con 4 modelos.
Generado: ensemble_avg\avg_fold4_norm_PCA95.csv con 4 modelos.
Generado: ensemble_avg\avg_fold5_norm_PCA95.csv con 4 modelos.
Generado: ensemble_avg\avg_fold1_original.csv con 4 modelos.
Generado: ensemble_avg\a

### Mediana

In [8]:
try:
    os.mkdir(path="ensemble_median")
except FileExistsError:
    pass

for set in SETS:
    for fold in range(1,6):
        
        prediction_dfs = []

        for method in METHODS:
            # Metemos en una lista todos los dfs de probabilidades para ese fold y set
            probabilities = obtener_probabilidad(method, fold, set)
            if not probabilities.empty:
                prediction_dfs.append(probabilities)

        # Creamos el DataFrame base
        df_median = pd.DataFrame(0, index=prediction_dfs[0].index, columns=CLASSES, dtype=int)

        # Concatemos los DataFrames para que sea más sencillo el cálculo de la mediana
        df_all_preds = pd.concat(prediction_dfs)

        # Agrupamos por el índice original (y calculamos la mediana
        df_median = df_all_preds.groupby(level=0)[CLASSES].median()
        df_median[CLASSES] = df_median[CLASSES].round(3)

        # Añadimos la columna que recoja la decisión del Ensemble
        df_median['Ensemble_Decision'] = df_median[CLASSES].idxmax(axis=1)

        # Guardamos en csv el DataFrame de votos
        file_name = f"median_fold{fold}_{set}.csv"
        full_path = os.path.join("ensemble_median", file_name)
            
        df_median.to_csv(full_path, index=True, index_label='Index')
        print(f"Generado: {full_path} con {len(prediction_dfs)} modelos.")        

Generado: ensemble_median\median_fold1_norm.csv con 4 modelos.
Generado: ensemble_median\median_fold2_norm.csv con 4 modelos.
Generado: ensemble_median\median_fold3_norm.csv con 4 modelos.
Generado: ensemble_median\median_fold4_norm.csv con 4 modelos.
Generado: ensemble_median\median_fold5_norm.csv con 4 modelos.
Generado: ensemble_median\median_fold1_norm_PCA80.csv con 4 modelos.
Generado: ensemble_median\median_fold2_norm_PCA80.csv con 4 modelos.
Generado: ensemble_median\median_fold3_norm_PCA80.csv con 4 modelos.
Generado: ensemble_median\median_fold4_norm_PCA80.csv con 4 modelos.
Generado: ensemble_median\median_fold5_norm_PCA80.csv con 4 modelos.
Generado: ensemble_median\median_fold1_norm_PCA95.csv con 4 modelos.
Generado: ensemble_median\median_fold2_norm_PCA95.csv con 4 modelos.
Generado: ensemble_median\median_fold3_norm_PCA95.csv con 4 modelos.
Generado: ensemble_median\median_fold4_norm_PCA95.csv con 4 modelos.
Generado: ensemble_median\median_fold5_norm_PCA95.csv con 4 mode